[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/worldbank/OpenNightLights/blob/master/onl/tutorials/mod6_3_intro_to_sentinel2.ipynb)

# Intro to Sentinel-2

<a href="https://sentinel.esa.int/web/sentinel/missions/sentinel-2">Sentinel-2 satellites</a> (there are 2) are part of the European Space Agency's (ESA) Copernicus system and contain the MultiSpectral Instrument (MSI), which provides hi-resolution imagery that is useful for land use monitoring.

As with other datasets in Google Earth Engine (GEE), more details and links to resources are available on the <a href="https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2">collection's landing pages</a>.

What is important for us is that it has several image bands across the optical electromagnetic spectrum that will be useful feature for classifying land use, particular that of built-up areas.

Let's explore this data source and get sense of it's resolution and attributes.

For visualization we'll look at just the Red, Green and Blue channels and clip around Nepal.

In [1]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [14]:
try:
        ee.Initialize(project='ee-ruslisan93')
except Exception as e:
        ee.Authenticate()
        ee.Initialize(project='ruslisan93')

# get our Nepal boundary
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME','Singapore')).geometry()

# Sentinel-2 image filtered on 2019 and on Nepal
se2 = ee.ImageCollection('COPERNICUS/S2_SR').filterDate("2019-01-01","2019-12-31").filterBounds(aoi).median().divide(10000)

rgb = ['B4','B3','B2']

# set some thresholds
rgbViz = {"min":0.0, "max":0.3,"bands":rgb}


# initialize our map
map1 = geemap.Map()
map1.centerObject(aoi, 7)
map1.addLayer(se2.clip(aoi), rgbViz, "S2")

map1.addLayerControl()
map1

Map(center=[1.361135578765903, 103.8202830927031], controls=(WidgetControl(options=['position', 'transparent_b…

We can see a real color image of Nepal. We reduced our Image Collection to an image representing the median of 2019 and it appears we've also captured some clouds around Kathmandu. We will make a cloud mask to clear the image up using Sentinel-2's QA band. We're modeling this (in Python) from the example used in GEE: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2#bands

In [ ]:
def se2mask(image):
    quality_band = image.select('QA60')

    # using the bit mask for clouds and cirrus clouds respectively
    cloudmask = 1 << 10
    cirrusmask = 1 << 11

    # we only want clear skies
    mask = quality_band.bitwiseAnd(cloudmask).eq(0) and (quality_band.bitwiseAnd(cirrusmask).eq(0))

    # we'll divide by 10000 to make interpreting the reflectance values easier
    return image.updateMask(mask).divide(10000)

se2 = ee.ImageCollection('COPERNICUS/S2').filterDate(
    "2019-01-01","2019-12-31").filterBounds(aoi).filter(
    ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",20)).map(se2mask).median()

# initialize our map
map2 = geemap.Map()
map2.centerObject(aoi, 7)
map2.addLayer(se2.clip(aoi), rgbViz, "S2")

map2.addLayerControl()
map2

Map(center=[28.268224502649357, 83.97637443188768], controls=(WidgetControl(options=['position', 'transparent_…

That cleared things up nicely!

Next we'll integrate Sentinel-2 image data with VIIRS-DNB nighttime lights.